# Text classification using Keras with Neptune tracking
Notebook inspired from https://keras.io/examples/nlp/text_classification_from_scratch/

## Setup

In [41]:
import os
import tensorflow as tf
import numpy as np

(Neptune) Import Neptune and initialize a project

In [42]:
os.environ["NEPTUNE_PROJECT"] = "showcase/project-text-classification"

In [43]:
import neptune.new as neptune

project = neptune.init_project()

https://app.neptune.ai/showcase/project-text-classification/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## Data preparation
We are using the IMDB sentiment analysis data available at https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz. For the purposes of this demo, we've uploaded this data to S3 at https://neptune-examples.s3.us-east-2.amazonaws.com/data/text-classification/aclImdb_v1.tar.gz and will be downloading it from there.

### (Neptune) Track datasets using Neptune
Since this dataset will be used among all the runs in the project, we track it at the project level

In [4]:
project["data/keras/files"].track_files(
    "s3://neptune-examples/data/text-classification/aclImdb_v1.tar.gz"
)
project.sync()

### (Neptune) Download files from S3 using Neptune

In [5]:
print("Downloading data...")
project["data/keras/files"].download("..")

### Extract downloaded files

In [6]:
import tarfile

print("Extracting data...")
my_tar = tarfile.open("../aclImdb_v1.tar.gz")
my_tar.extractall("..")
my_tar.close()

Extracting data...


### Remove `unsup` subfolder and rename `aclImdb` folder

In [7]:
import shutil

shutil.rmtree("../aclImdb/train/unsup")
os.remove("../aclImdb_v1.tar.gz")

if os.path.exists("../data"):
    shutil.rmtree("../data")
os.rename("../aclImdb", "../data")

(Neptune) Upload dataset sample to Neptune project

In [8]:
import random

project["data/keras/sample/train/pos"].upload(
    f"../data/train/pos/{random.choice(os.listdir('../data/train/pos'))}"
)
project["data/keras/sample/train/neg"].upload(
    f"../data/train/neg/{random.choice(os.listdir('../data/train/neg'))}"
)
project["data/keras/sample/test/pos"].upload(
    f"../data/test/pos/{random.choice(os.listdir('../data/test/pos'))}"
)
project["data/keras/sample/test/neg"].upload(
    f"../data/test/neg/{random.choice(os.listdir('../data/test/neg'))}"
)

### Generate training, validation, and test datasets

In [44]:
data_params = {
    "batch_size": 32,
    "validation_split": 0.2,
    "max_features": 20000,
    "embedding_dim": 128,
    "sequence_length": 500,
    "seed": 42,
}

(Neptune) Log data metadata to Neptune

In [45]:
run = neptune.init_run(name="Keras text classification", tags=["keras"])

https://app.neptune.ai/showcase/project-text-classification/e/TXTCLF-352
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [46]:
run["data/params"] = data_params

In [47]:
from tensorflow.keras.layers import TextVectorization
import string
import re

In [48]:
raw_train_ds, raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/train",
    batch_size=data_params["batch_size"],
    validation_split=data_params["validation_split"],
    subset="both",
    seed=data_params["seed"],
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/test", batch_size=data_params["batch_size"]
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


### Previewing data

In [49]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
0
b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into complicated situatio

### Clean data

In [50]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=data_params["max_features"],
    output_mode="int",
    output_sequence_length=data_params["sequence_length"],
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### Vectorize data

In [51]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Modelling

(Neptune) Create a new model and model version

In [52]:
from neptune.new.exceptions import NeptuneModelKeyAlreadyExistsError

project_key = project.get_structure()["sys"]["id"].fetch()

try:
    model = neptune.init_model(name="keras", key="KER")
    model.stop()
    model_version = neptune.init_model_version(model=f"{project_key}-KER", name="keras")
except NeptuneModelKeyAlreadyExistsError:
    model_version = neptune.init_model_version(model=f"{project_key}-KER", name="keras")

https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/v/TXTCLF-KER-4
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


### Build a model

In [53]:
model_params = {
    "dropout": 0.5,
    "strides": 3,
    "activation": "relu",
    "kernel_size": 7,
    "loss": "binary_crossentropy",
    "optimizer": "adam",
    "metrics": ["accuracy"],
}

In [54]:
model_version["params"] = model_params

In [55]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(data_params["max_features"], data_params["embedding_dim"])(inputs)
x = layers.Dropout(model_params["dropout"])(x)

# Conv1D + global max pooling
x = layers.Conv1D(
    data_params["embedding_dim"],
    model_params["kernel_size"],
    padding="valid",
    activation=model_params["activation"],
    strides=model_params["strides"],
)(x)
x = layers.Conv1D(
    data_params["embedding_dim"],
    model_params["kernel_size"],
    padding="valid",
    activation=model_params["activation"],
    strides=model_params["strides"],
)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(data_params["embedding_dim"], activation=model_params["activation"])(x)
x = layers.Dropout(model_params["dropout"])(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

keras_model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
keras_model.compile(
    loss=model_params["loss"], optimizer=model_params["optimizer"], metrics=model_params["metrics"]
)

### Train the model

(Neptune) Initialize the Neptune callback

In [56]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

neptune_callback = NeptuneCallback(run=run, log_model_diagram=True, log_on_batch=True)

In [57]:
training_params = {
    "epochs": 3,
}

In [58]:
# Fit the model using the train and test datasets.
keras_model.fit(
    train_ds, validation_data=val_ds, epochs=training_params["epochs"], callbacks=neptune_callback
)
# Training parameters are logged automatically to Neptune

Epoch 1/3
625/625 [==============================] - 47s 73ms/step - loss: 0.5012 - accuracy: 0.7154 - val_loss: 0.3190 - val_accuracy: 0.8640
Epoch 2/3
625/625 [==============================] - 39s 63ms/step - loss: 0.2248 - accuracy: 0.9122 - val_loss: 0.3073 - val_accuracy: 0.8822
Epoch 3/3
625/625 [==============================] - 41s 66ms/step - loss: 0.1112 - accuracy: 0.9599 - val_loss: 0.4393 - val_accuracy: 0.8692


### Evaluate the model

In [59]:
_, acc = keras_model.evaluate(test_ds, callbacks=neptune_callback)

782/782 [==============================] - 14s 18ms/step - loss: 0.4816 - accuracy: 0.8529


## (Neptune) Associate run with model and vice-versa

In [60]:
run_meta = {
    "id": run.get_structure()["sys"]["id"].fetch(),
    "name": run.get_structure()["sys"]["name"].fetch(),
    "url": run.get_url(),
}
run_meta

{'id': 'TXTCLF-352',
 'name': 'Keras text classification',
 'url': 'https://app.neptune.ai/showcase/project-text-classification/e/TXTCLF-352'}

In [61]:
model_version["run"] = run_meta

In [62]:
model_version_meta = {
    "id": model_version.get_structure()["sys"]["id"].fetch(),
    "name": model_version.get_structure()["sys"]["name"].fetch(),
    "url": model_version.get_url(),
}
model_version_meta

{'id': 'TXTCLF-KER-4',
 'name': 'keras',
 'url': 'https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/v/TXTCLF-KER-4'}

In [63]:
run["training/model/meta"] = model_version_meta

## (Neptune) Upload serialized model and model weights to Neptune

In [64]:
model_version["serialized_model"] = keras_model.to_json()

In [65]:
keras_model.save_weights("model_weights.h5")
model_version["model_weights"].upload("model_weights.h5")

(Neptune) Wait for all operations to sync with Neptune servers

In [66]:
model_version.sync()

## (Neptune) Promote best model to production

### (Neptune) Fetch current production model

In [67]:
model = neptune.init_model(with_id=f"{project_key}-KER")
model_versions_df = model.fetch_model_versions_table().to_pandas()
model.stop()
model_versions_df

https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER
Remember to stop your model once you’ve finished logging your metadata (https://docs.neptune.ai/api/model#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/metadata


,sys/creation_time,sys/id,sys/model_id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,...,params/dropout,params/kernel_size,params/loss,params/metrics,params/optimizer,params/strides,run/id,run/name,run/url,serialized_model
0,2023-01-05 18:00:54.375000+00:00,TXTCLF-KER-4,TXTCLF-KER,2023-01-05 18:03:18.001000+00:00,139,keras,siddhant.sadangi,2023-01-05 18:03:15.671000+00:00,141.292,333.0,...,0.5,7,binary_crossentropy,['accuracy'],adam,3,NaN,NaN,NaN,NaN
1,2023-01-05 17:57:35.457000+00:00,TXTCLF-KER-3,TXTCLF-KER,2023-01-05 18:00:09.814000+00:00,137,keras,siddhant.sadangi,2023-01-05 18:00:09.814000+00:00,154.349,11255453.0,...,0.5,7,binary_crossentropy,['accuracy'],adam,3,TXTCLF-350,Keras text classification,https://app.neptune.ai/showcase/project-text-c...,"{""class_name"": ""Functional"", ""config"": {""name""..."
2,2023-01-05 17:35:27.859000+00:00,TXTCLF-KER-2,TXTCLF-KER,2023-01-05 18:00:09.752000+00:00,123,Untitled,siddhant.sadangi,2023-01-05 18:03:12.457000+00:00,1664.514,11255454.0,...,0.5,7,binary_crossentropy,['accuracy'],adam,3,TXTCLF-348,Keras text classification,https://app.neptune.ai/showcase/project-text-c...,"{""class_name"": ""Functional"", ""config"": {""name""..."
3,2023-01-05 17:26:18.799000+00:00,TXTCLF-KER-1,TXTCLF-KER,2023-01-05 17:39:34.330000+00:00,49,Untitled,siddhant.sadangi,2023-01-05 18:03:13.281000+00:00,1569.103,4529.0,...,0.5,7,binary_crossentropy,['accuracy'],adam,3,NaN,NaN,NaN,"{""class_name"": ""Functional"", ""config"": {""name""..."


In [68]:
production_models = model_versions_df[model_versions_df["sys/stage"] == "production"]["sys/id"]
assert (
    len(production_models) == 1
), f"Multiple model versions found in production: {production_models.values}"

In [69]:
prod_model_id = production_models.values[0]
print(f"Current model in production: {prod_model_id}")

Current model in production: TXTCLF-KER-3


In [70]:
prod_model = neptune.init_model_version(with_id=prod_model_id)
prod_model_params = prod_model["params"].fetch()
loaded_prod_model = tf.keras.models.model_from_json(
    prod_model["serialized_model"].fetch(), custom_objects=None
)

prod_model["model_weights"].download()
loaded_prod_model.load_weights("model_weights.h5")

https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/v/TXTCLF-KER-3
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


### (Neptune) Evaluate current model on lastest test data

In [71]:
# using the model's original loss and optimizer, but the current metric
loaded_prod_model.compile(
    loss=prod_model_params["loss"],
    optimizer=prod_model_params["optimizer"],
    metrics=model_params["metrics"],
)

_, prod_model_acc = loaded_prod_model.evaluate(test_ds)

782/782 [==============================] - 11s 13ms/step - loss: 0.4290 - accuracy: 0.8634


### (Neptune) If challenger model outperforms production model, promote it to production

In [72]:
print(f"Production model accuracy: {prod_model_acc}\nChallenger model accuracy: {acc}")

if acc > prod_model_acc:
    print("Promoting challenger to production")
    prod_model.change_stage("archived")
    model_version.change_stage("production")
else:
    print("Archiving challenger model")
    model_version.change_stage("archived")

prod_model.stop()

Production model accuracy: 0.8633599877357483
Challenger model accuracy: 0.8529199957847595
Archiving challenger model
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/v/TXTCLF-KER-3/metadata


## (Neptune) Stop tracking

In [73]:
model_version.stop()
run.stop()
project.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/showcase/project-text-classification/m/TXTCLF-KER/v/TXTCLF-KER-4/metadata
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 96 operations to synchronize with Neptune. Do not kill this process.
All 96 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/showcase/project-text-classification/e/TXTCLF-352
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/showcase/project-text-classification/metadata
